In [3]:
from langchain.chat_models import ChatOpenAI
from langchain.document_loaders import UnstructuredFileLoader
from langchain.text_splitter import CharacterTextSplitter
from langchain.embeddings import OpenAIEmbeddings, CacheBackedEmbeddings
from langchain.vectorstores import FAISS
from langchain.storage import LocalFileStore
from langchain.prompts import ChatPromptTemplate
from langchain.schema.runnable import RunnablePassthrough

In [5]:
llm = ChatOpenAI(
    temperature = 0.1
)


splitter = CharacterTextSplitter.from_tiktoken_encoder(
    separator="\n",
    chunk_size=600,
    chunk_overlap=100,
)


# txt,doc,pdf.html,excel 등등 다양한 file들을 load 할 수 있다.
loader = UnstructuredFileLoader("../KAIA-JKIICE.pdf")

docs = loader.load_and_split(text_splitter=splitter)

# embedding은 고ㅇ

cache_dir = LocalFileStore("./.cache/")

# Embedding은 text에 의미별로 적절한 점수를 부여해서 vector 형식으로 표현한것,

embeddings = OpenAIEmbeddings()

# ChacheBackedEmbedding을 사용하여 만들어진 Embedding을 cache(저장) 한다.

cached_embeddings = CacheBackedEmbeddings.from_bytes_store(embeddings,cache_dir)

# Vector store를 호출 해주고 FAISS사용
# from_documents 메서드는 document 별로 embedding 작업 후 결과를 저장한 vector store를 반환
# 이를 이용해 document 검색도 하고, 연관성이 높은 document들을 찾는다.
vectorstore = FAISS.from_documents(docs,cached_embeddings)

# retriver는 document들의 list를 반환
retriver = vectorstore.as_retriever()

prompt = ChatPromptTemplate.from_messages([
    ("system","""You are a helpful assistant. Answer questions using only the following context.
     If you don't know the answer just say you don't, don't make it up:\n\n{context}"""),
     ("human","{question}")
])

# RunnablePassthrough 말그대로 통과게 해준다.("이 논문은 무엇을 하는 논문인지 설명해주고, 한글로 해석해서 알려줘." 
# 이 입력값이 RunnablePassthrough를 통해서 question에 전달된다.)

chain = {"context":retriver, "question" : RunnablePassthrough() } | prompt | llm


chain.invoke("이 논문은 무엇을 하는 논문인지 설명해주고, 한글로 해석해서 알려줘.")

Created a chunk of size 1029, which is longer than the specified 600
Created a chunk of size 864, which is longer than the specified 600
Created a chunk of size 643, which is longer than the specified 600
Created a chunk of size 636, which is longer than the specified 600
Created a chunk of size 726, which is longer than the specified 600
Created a chunk of size 918, which is longer than the specified 600
Created a chunk of size 692, which is longer than the specified 600
Created a chunk of size 1431, which is longer than the specified 600


AIMessage(content='이 논문은 열화상 카메라와 광각 카메라를 동시에 적용하여 YOLO 기반의 딥러닝 알고리즘을 이용한 데이터 분석을 기반으로 도로의 블랙아이스, 포트홀 및 장애물을 실시간으로 탐지하며, 도로 사용자에게 실시간으로 중요한 안전 정보를 제공하는 실용적이고 효율적인 도로 안전 시스템을 개발하는 것이다.')